In [9]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
import time
from scipy.ndimage import map_coordinates
from skimage.transform import resize

import sys
sys.path.append('../common/')

from image_commons import *
from utils import *
from optimisation import *
from loo_cv import *
from rbf_tools import *
from two_dim import *

In [10]:
seed=0
torch.manual_seed(seed)

This notebooks computes the test error, the 1d interpolatione error on 3 test cases and the 2d interpolation error in 3 test cases, both in uniform mesh and non-uniform mesh.

In [11]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [12]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# generate features

In [13]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [14]:
# network to be tested:
directory = "plots/"
if not os.path.exists(directory):
    os.makedirs(directory)

# network to be tested:
n=10
path =   '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=45, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)

# Upscale two dimension 

In [15]:
def upscale_image(image, downscaled_image):
    original_shape = image.shape

    # Prepare data for RBF interpolation
    # interpolation points
    x = np.linspace(0, original_shape[0] - 1, downscaled_image.shape[0])
    y = np.linspace(0, original_shape[1] - 1, downscaled_image.shape[1])
    xv, yv = np.meshgrid(x, y)
    coords = np.vstack([xv.ravel(), yv.ravel()]).T  # Create a list of coordinates
    
    values = downscaled_image.ravel()  # Flatten grayscale values

    #evaluation points
    x_full = np.linspace(0, original_shape[0] - 1, original_shape[0])
    y_full = np.linspace(0, original_shape[1] - 1, original_shape[1])
    xv_full, yv_full = np.meshgrid(x_full, y_full)
    coords_full = np.vstack([xv_full.ravel(), yv_full.ravel()]).T

    p_ref=np.array([0,0])    
    no_evaluation_points=coords_full.shape[0]
    distance =cdist(coords_full,coords)
    closest = np.argpartition(distance, 10, axis=1)
    scaled_training_set_flat = []
    eps_final_hardy=np.zeros((no_evaluation_points,1))
    eps_final_franke=np.zeros((no_evaluation_points,1))
    eps_final_mfranke=np.zeros((no_evaluation_points,1))
    eps_final_rippa=np.zeros((no_evaluation_points,1))
    eps_final_mle=np.zeros((no_evaluation_points,1))
    
    x_coords = []
    f=[]
    upscaled_flat=np.zeros((6,no_evaluation_points))
    print(no_evaluation_points)
    for i in range(no_evaluation_points):
        x_local=np.zeros((10,2))
        fpart=np.zeros((10+1,1))
        for j in range(10):
            x_local[j]=coords[closest[i][j]]
            fpart[j]=values[closest[i][j]]
        x_local = sorted( x_local, key = lambda x: np.linalg.norm(x - p_ref ) )
        x_local = np.reshape(x_local, (10,2)) 
        x_coords.append(x_local)
        f.append(fpart)
        x_axis=x_local.reshape(10,2)
        # Hardy approach
        nearest_distances = find_nearest_neighbors(x_axis)
        d=np.sum(nearest_distances)/n        
        eps_final_hardy[i]=1/(0.815*d)

        # Franke approach
        x_axis_list=x_axis.tolist()        
        circle = find_min_circle(x_axis_list)
        eps_final_franke[i]=0.8*(10**(1/2))/(2 * circle[2])

        # Modified Franke approach
        eps_final_mfranke[i]=0.8*(10**(1/4))/(2 * circle[2])

        # Rippa approach       
        rhs_rippa = fpart[:10]
        eps_v = [0.001, 0.002, 0.005, 0.0075, 0.01, 0.02, 0.05, \
                 0.075, 0.1, 0.2, 0.5, 0.75,1,2, 5, 7.5, 10, 20.0, 50, 75, 100, 200, 500, 1000]
        #
        best_eps = 0
        old_error = np.inf        
        for eps in eps_v:
            error_r = rippa_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_rippa[i]=best_eps

        ## MLE approach
        rhs_rippa = fpart[:10]
        #eps_v = #np.concatenate([np.linspace(0.001,30,200),[50.0, 75.0, 100.0, 200.0, 500.0, 1000.0]]) 
        best_eps = 1.0
        old_error = np.inf        
        for eps in eps_v:
            error_r = mle_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_mle[i]=best_eps
        #print(best_eps)
        
        # NN approach
        xxx=generate_distance_from_coordinates(x_axis)               
        training_set_distances_flatten = upper_half(xxx)
        scaled_training_set_flat.append(training_set_distances_flatten)
        if i%1000==0:
            print(i)

    
    scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)
    scaled_training_set_tensor.reshape((no_evaluation_points,int(10*10/2-10/2)))
        
    eps_final= model(scaled_training_set_tensor)
    eps_final_optimisation = eps_final.detach().numpy()
    print(eps_final_optimisation)
    for j in range(no_evaluation_points):
        for count,eps in enumerate([eps_final_mle[j][0],eps_final_rippa[j][0],eps_final_hardy[j][0],eps_final_franke[j][0],eps_final_mfranke[j][0],eps_final_optimisation[j][0]]):
            L =  get_int_matrix(x_coords[j],eps)                
            fx=f[j]
           
            w=np.linalg.solve(L, fx)             
            
            s=np.zeros((1,10+1))
            for i in range(10):      
                s[0,i] = phi(eps,coords_full[j] , x_coords[j][i])
            s[0,-1]=1.0        
            
            upscaled_flat[count][j]=np.matmul(s,w)          
        
    upscaled_image=[]
    for count in range(6):        
        upscaled_image.append(upscaled_flat[count].reshape(original_shape[:2]))
    return upscaled_image

# Visualize the images
def display_images(upscaled_image,file_name):
    
    fig = plt.figure(figsize=(10, 6))
    plt.imshow(upscaled_image, cmap='gray')
    plt.axis('off')
    plt.savefig(f'upscaled_'+file_name,bbox_inches='tight', dpi=150)
    plt.close(fig)

# Checkerboard

In [ ]:
image = create_checkerboard(size=(200, 200), num_checks=10)  # Create a synthetic checkerboard image
original_shape = image.shape

downscale_factor = 0.5
downscaled_image = resize(image, (int(original_shape[0] * downscale_factor),
                                  int(original_shape[1] * downscale_factor)))

upscaled_image = upscale_image(image, downscaled_image)  # upscale the image

fig1 = plt.figure(figsize=(10, 6))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.savefig(f'original',bbox_inches='tight', dpi=150)
plt.close(fig1)

fig2 = plt.figure(figsize=(5, 3))
plt.imshow(downscaled_image, cmap='gray')
plt.axis('off')
plt.savefig(f'downscaled',bbox_inches='tight', dpi=150)
plt.close(fig2)

strategies=['MLE','Rippa','Hardy','Franke','Modified Franke', 'NN']

mse=[0 for i in range(6)]
psnr=[0 for i in range(6)]

for count in range(6):
    display_images(upscaled_image[count],f"{strategies[count]}_upscale_checkerboard")  # Display the images
    mse[count] = np.mean((image - upscaled_image[count]) ** 2)
    psnr[count]=20*np.log10(np.max(image)/np.sqrt(mse[count]))

print('mse',mse)
print('psnr',psnr)

40000


/users/PAS2708/hanveiga1/new_rbf/RBF-shape-parameter-NN/2-dimensional/../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)


0


# Pepper

In [10]:
path = 'flower.png'
image = load_image(path) 
original_shape = image.shape

downscale_factor = 0.5
downscaled_image = resize(image, (int(original_shape[0] * downscale_factor),
                                  int(original_shape[1] * downscale_factor)))

upscaled_image = upscale_image(image, downscaled_image)  # upscale the image

fig1 = plt.figure(figsize=(10, 6))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.savefig(f'original_flower',bbox_inches='tight', dpi=150)
plt.close(fig1)

fig2 = plt.figure(figsize=(5, 3))
plt.imshow(downscaled_image, cmap='gray')
plt.axis('off')
plt.savefig(f'downscaled_flower',bbox_inches='tight', dpi=150)
plt.close(fig2)

strategies=['MLE','Rippa','Hardy','Franke','Modified Franke', 'NN']

mse=[0 for i in range(6)]
psnr=[0 for i in range(6)]

for count in range(6):
    display_images(upscaled_image[count],strategies[count]+'_upscale_flower')  # Display the images
    mse[count] = np.mean((image - upscaled_image[count]) ** 2)
    psnr[count]=20*np.log10(np.max(image)/np.sqrt(mse[count]))

print('mse',mse)
print('psnr',psnr)

40000
0


/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/2-dimensional/image/../../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000


/tmp/slurmtmp.672334/ipykernel_3483287/4005633290.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/torch/csrc/utils/tensor_new.cpp:245.)
  scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)


[[2.2928782]
 [2.2983208]
 [2.2927067]
 ...
 [2.2914257]
 [2.2910979]
 [2.2931643]]
mse [0.007672205671218552, 0.007670825120523483, 0.007472746263846776, 0.007861158756381164, 0.007854443495780206, 0.005167534164862738]
psnr [20.909027641977683, 20.90980918971569, 21.02342764692147, 20.803364338277103, 20.807075810168843, 22.625396441620587]
